In [11]:
!pip install pylogit

     |████████████████████████████████| 153kB 9.8MB/s 


In [12]:
import pandas as pd
import numpy as np
import pylogit
from scipy.special import logit
pd.options.display.max_columns = 100

In [3]:
df = pd.read_csv('xc.csv', parse_dates=['dt'])

In [4]:
dups = df.groupby('race_id').box.agg(lambda x : len(x)-x.nunique())
dups = dups.loc[dups>0]
df = df.loc[~df.race_id.isin(dups.index)].copy()

In [5]:
win_choice = df.loc[df.winner == 1].sample(frac = 1.0).groupby('race_id').head(1).loc[:,['race_id','box']].copy()
win_choice['twinner'] = 1
win_choice.head()

,race_id,box,twinner
382947,4603695,1,1
167722,4511250,3,1
163412,4510520,5,1
371091,4598054,5,1
85983,4459924,2,1


In [6]:
dfm = df.merge(win_choice, on = ['race_id','box'], how = 'left')
dfm.twinner = dfm.twinner.fillna(0)
dfm = dfm.sort_values(['race_id','box'])
dfm.head()

,Unnamed: 0,race_id,stadium_id,distance_m,going,race_type,race_grade,dog_id,place,time,decimal_price,comment,box,kg,winner,dt,origTime,dnf,twinner
1,2,4418717,13025,515,-0.1,flat_race,A6,2207348,3,31.33,7.00,"CrdRnUp&1,Led2To 3/4",1,31.25,0,2019-01-01,31.33,0,0.0
5,6,4418717,13025,515,-0.1,flat_race,A6,2354055,1,31.04,2.75,"CrdRnUp&1,Mid 1/2,Ld 3/4",2,28.75,1,2019-01-01,31.04,0,1.0
4,5,4418717,13025,515,-0.1,flat_race,A6,2342199,6,31.70,4.00,"EP,CrdRnUp&1& 1/4&3&4",3,34.00,0,2019-01-01,31.70,0,0.0
3,4,4418717,13025,515,-0.1,flat_race,A6,2338001,2,31.31,3.50,"SAw,BBlk 1/4,Crowded3",4,35.50,0,2019-01-01,31.31,0,0.0
0,1,4418717,13025,515,-0.1,flat_race,A6,2176330,4,31.41,11.00,"BadlyBlkVW1,Crd3",5,27.75,0,2019-01-01,31.41,0,0.0


In [7]:
startupTime = 1.75
distDft = 400
distExp = 0.11
dfm['runTime'] = dfm.time - startupTime
dfm['mmps'] = dfm.distance_m/dfm.runTime * (dfm.distance_m**distExp)/(distDft**distExp)

In [8]:
dfm['avg_mmps'] = dfm.groupby(['dog_id']).mmps.transform(lambda x : x.expanding().mean().shift(1, fill_value = 0))

In [13]:
pylogit.choice_calcs.__file__

'/usr/local/lib/python3.7/dist-packages/pylogit/choice_calcs.py'

In [9]:
def mlogit(formula, df, obs_id, alt_id) :
    """
    df must be sorted by obs_id to use this function
    """
    from patsy import dmatrices
    from collections import OrderedDict
    import pylogit as pl
    import numpy as np
    data = dmatrices(formula+' -1', df, return_type = "dataframe")
    all_alts = sorted(df.loc[:,alt_id].unique())
    
    spec = OrderedDict()
    names = OrderedDict()
    spec['intercept'] = all_alts[1:]
    names['intercept'] = [f'(Intercept):{alt}' for alt in all_alts[1:]]
    c_names = data[1].design_info.column_names
    choice = data[0].design_info.column_names[0]
    data[1][choice] = data[0]
    data[1][[obs_id,alt_id]] = df[[obs_id,alt_id]]    
    for c in c_names :
        spec[c] = [all_alts]
        names[c] = [c]
    model = pl.create_choice_model(data = data[1], alt_id_col = alt_id, obs_id_col = obs_id, 
                                   choice_col = choice, specification = spec, names = names, model_type = 'MNL')
    model.fit_mle(np.zeros(model.design.shape[1]), method = 'Powell')
    return model

In [10]:
model = mlogit('twinner ~ avg_mmps', dfm, 'race_id', 'box')

ModuleNotFoundError: ignored

In [ ]:
model.get_statsmodels_summary()